In [20]:
import tensorflow as tf
import shutil
import math
from datetime import datetime
import multiprocessing
from tensorflow.python.feature_column import feature_column
import schema
sess=tf.Session()
print(tf.__version__)

1.12.0


In [3]:
MODEL_NAME = 'reg-model-1_2'

TRAIN_DATA_FILES_PATTERN = 'data/train-*.csv'
VALID_DATA_FILES_PATTERN = 'data/valid-*.csv'
TEST_DATA_FILES_PATTERN = 'data/test-*.csv'

RESUME_TRAINING = False
PROCESS_FEATURES = True
MULTI_THREADING = True

In [5]:
HEADER = ['key','x','y','alpha','beta','target']
HEADER_DEFAULTS = [[0], [0.0], [0.0], ['NA'], ['NA'], [0.0]]
NUMERIC_FEATURE_NAMES = ['x', 'y']  
CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY = {'alpha':['ax01', 'ax02'], 'beta':['bx01', 'bx02']}
CATEGORICAL_FEATURE_NAMES = list(CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY.keys())
FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES
TARGET_NAME = 'target'
UNUSED_FEATURE_NAMES = list(set(HEADER) - set(FEATURE_NAMES) - {TARGET_NAME})
print("Header: {}".format(HEADER))
print("Numeric Features: {}".format(NUMERIC_FEATURE_NAMES))
print("Categorical Features: {}".format(CATEGORICAL_FEATURE_NAMES))
print("Target: {}".format(TARGET_NAME))
print("Unused Features: {}".format(UNUSED_FEATURE_NAMES))

Header: ['key', 'x', 'y', 'alpha', 'beta', 'target']
Numeric Features: ['x', 'y']
Categorical Features: ['alpha', 'beta']
Target: target
Unused Features: ['key']


In [45]:
def parse_csv_row(csv_row):
    """get a string tensor"""
    columns = tf.decode_csv(csv_row,record_defaults=HEADER_DEFAULTS)
    features = dict(zip(HEADER,columns))
    for column in UNUSED_FEATURE_NAMES:
        features.pop(column)
    target = features.pop(TARGET_NAME)
    return features, target

def process_features(features):
    features["x_2"] = tf.square(features['x'])
    features["y_2"] = tf.square(features['y'])
    features["xy"] = tf.multiply(features['x'], features['y']) # features['x'] * features['y']
    features['dist_xy'] =  tf.sqrt(tf.squared_difference(features['x'],features['y']))
    return features

def csv_input_fn(file_name_pattern, mode=tf.estimator.ModeKeys.EVAL, 
                 skip_header_lines=0, num_epochs=None, batch_size=200):
    
    shuffle = True if mode == tf.estimator.ModeKeys.TRAIN else False
    input_file_names = tf.matching_files(pattern=file_name_pattern)
    dataset = tf.data.TextLineDataset(input_file_names)
    dataset = dataset.skip(skip_header_lines)
    dataset = dataset.map(parse_csv_row)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=2 * batch_size + 1)
    if PROCESS_FEATURES:
        dataset = dataset.map(lambda features, target: (process_features(features), target))
    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat(num_epochs)
    iterator = dataset.make_one_shot_iterator()
    feaures,target = iterator.get_next()
    return feaures,target   

In [47]:
CONSTRUCTED_NUMERIC_FEATURES_NAMES = ['x_2', 'y_2', 'xy', 'dist_xy']


all_numeric_feature_names = NUMERIC_FEATURE_NAMES + CONSTRUCTED_NUMERIC_FEATURES_NAMES

numeric_columns = {feature_name: tf.feature_column.numeric_column(feature_name)
                       for feature_name in all_numeric_feature_names}

In [52]:

categorical_column_with_vocabulary = {item[0]: tf.feature_column.categorical_column_with_vocabulary_list(item[0], item[1])
         for item in CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY.items()}

In [54]:
    SERVING_HEADER = ['x', 'y', 'alpha', 'beta']
    SERVING_HEADER_DEFAULTS = [[0.0], [0.0], ['NA'], ['NA']]
    rows_string_tensor = tf.placeholder(dtype=tf.string, shape=[None], name='csv_rows')

    receiver_tensor = {'csv_rows': rows_string_tensor}

    row_columns = tf.expand_dims(rows_string_tensor, -1)
    columns = tf.decode_csv(row_columns, record_defaults=SERVING_HEADER_DEFAULTS)
    features = dict(zip(SERVING_HEADER, columns))

    if PROCESS_FEATURES:
        features = process_features(features)

    tf.estimator.export.ServingInputReceiver(features, receiver_tensor)

ServingInputReceiver(features={'x': <tf.Tensor 'DecodeCSV:0' shape=(?, 1) dtype=float32>, 'y': <tf.Tensor 'DecodeCSV:1' shape=(?, 1) dtype=float32>, 'alpha': <tf.Tensor 'DecodeCSV:2' shape=(?, 1) dtype=string>, 'beta': <tf.Tensor 'DecodeCSV:3' shape=(?, 1) dtype=string>, 'x_2': <tf.Tensor 'Square:0' shape=(?, 1) dtype=float32>, 'y_2': <tf.Tensor 'Square_1:0' shape=(?, 1) dtype=float32>, 'xy': <tf.Tensor 'Mul:0' shape=(?, 1) dtype=float32>, 'dist_xy': <tf.Tensor 'Sqrt:0' shape=(?, 1) dtype=float32>}, receiver_tensors={'csv_rows': <tf.Tensor 'csv_rows:0' shape=(?,) dtype=string>}, receiver_tensors_alternatives=None)

In [55]:
type(tf.estimator.export.ServingInputReceiver(features, receiver_tensor))

tensorflow.python.estimator.export.export.ServingInputReceiver

In [ ]:
import tensorflow as tf
import shutil
import math
from datetime import datetime
import multiprocessing
from tensorflow.python.feature_column import feature_column
import schema

print(tf.__version__)
sess = tf.Session()

MODEL_NAME = 'reg-model-1_2'

TRAIN_DATA_FILES_PATTERN = 'data/train-*.csv'
VALID_DATA_FILES_PATTERN = 'data/valid-*.csv'
TEST_DATA_FILES_PATTERN = 'data/test-*.csv'

RESUME_TRAINING = False
PROCESS_FEATURES = True
MULTI_THREADING = True

HEADER = ['key', 'x', 'y', 'alpha', 'beta', 'target']
HEADER_DEFAULTS = [[0], [0.0], [0.0], ['NA'], ['NA'], [0.0]]
NUMERIC_FEATURE_NAMES = ['x', 'y']
CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY = {'alpha':['ax01', 'ax02'], 'beta':['bx01', 'bx02']}
CATEGORICAL_FEATURE_NAMES = list(CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY.keys())
FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES
TARGET_NAME = 'target'
UNUSED_FEATURE_NAMES = list(set(HEADER) - set(FEATURE_NAMES) - {TARGET_NAME})
print("Header: {}".format(HEADER))
print("Numeric Features: {}".format(NUMERIC_FEATURE_NAMES))
print("Categorical Features: {}".format(CATEGORICAL_FEATURE_NAMES))
print("Target: {}".format(TARGET_NAME))
print("Unused Features: {}".format(UNUSED_FEATURE_NAMES))

from typing import ByteString, Dict, Tuple, Optional, Any


def parse_csv_row(csv_row: ByteString)->Tuple[Dict[str, tf.Tensor], tf.Tensor]:
    """get a string tensor"""
    columns = tf.decode_csv(csv_row, record_defaults=HEADER_DEFAULTS)
    features: Dict[str, tf.Tensor] = dict(zip(HEADER, columns))
    for column in UNUSED_FEATURE_NAMES:
        features.pop(column)
    target = features.pop(TARGET_NAME)
    return features, target


def process_features(features: Dict[str, tf.Tensor])->Dict[str, tf.Tensor]:
    features["x_2"] = tf.square(features['x'])
    features["y_2"] = tf.square(features['y'])
    features["xy"] = tf.multiply(features['x'], features['y'])  # features['x'] * features['y']
    features['dist_xy'] = tf.sqrt(tf.squared_difference(features['x'], features['y']))
    return features


def csv_input_fn(file_name_pattern: str, mode: str=tf.estimator.ModeKeys.EVAL, batch_size: int=200,
                 num_epochs: Optional[int]=None, skip_header_lines: int=0)->Tuple[Dict[str, tf.Tensor], tf.Tensor]:
    shuffle = True if mode == tf.estimator.ModeKeys.TRAIN else False
    input_file_names: tf.Tensor = tf.matching_files(pattern=file_name_pattern)
    dataset = tf.data.TextLineDataset(input_file_names)
    dataset = dataset.skip(skip_header_lines)
    dataset = dataset.map(parse_csv_row)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=2 * batch_size + 1)
    if PROCESS_FEATURES:
        dataset = dataset.map(lambda features, target: (process_features(features), target))
    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat(num_epochs)
    iterator = dataset.make_one_shot_iterator()
    feaures, target = iterator.get_next()
    return feaures, target


CONSTRUCTED_NUMERIC_FEATURES_NAMES = ['x_2', 'y_2', 'xy', 'dist_xy']


def get_feature_columns()->Dict[str, Any]:
    # 将各种列赋予类型，根据类型可以有响应处理
    all_numeric_feature_names = NUMERIC_FEATURE_NAMES + CONSTRUCTED_NUMERIC_FEATURES_NAMES
    numeric_columns: Dict[str, Any] = {
        feature_name: tf.feature_column.numeric_column(feature_name) for feature_name in all_numeric_feature_names}

    categorical_column_with_vocabulary: Dict[str, Any] = {
        item[0]: tf.feature_column.categorical_column_with_vocabulary_list(item[0], item[1])
        for item in CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY.items()}
    feature_columns = {}
    if numeric_columns is not None:
        feature_columns.update(numeric_columns)

    if categorical_column_with_vocabulary is not None:
        feature_columns.update(categorical_column_with_vocabulary)
    # 之所用用字典，方便后面按照名称索引进一步处理，因为列还没有进一步转化
    feature_columns['alpha_X_beta'] = tf.feature_column.crossed_column([feature_columns['alpha'], feature_columns['beta']], 4)

    return feature_columns


print("Feature Columns: {}".format(get_feature_columns()))


def create_estimator(run_config, hparams)->tf.estimator.Estimator:
    # 检查字段并对字段做按类别做出进一步处理
    FEATURE_COLUMNS = list(get_feature_columns().values())
    dense_columns = list(filter(lambda column: isinstance(column, feature_column._NumericColumn),FEATURE_COLUMNS))

    categorical_columns = list(
        filter(lambda column: isinstance(column, feature_column._VocabularyListCategoricalColumn) |
                              isinstance(column, feature_column._BucketizedColumn), FEATURE_COLUMNS))

    # convert categorical columns to indicators,独热码化
    indicator_columns = list( map(lambda column: tf.feature_column.indicator_column(column), categorical_columns))

    estimator = tf.estimator.DNNRegressor(
        feature_columns=dense_columns + indicator_columns,
        hidden_units=hparams.hidden_units,
        optimizer=tf.train.AdamOptimizer(),
        activation_fn=tf.nn.elu,
        dropout=hparams.dropout_prob,
        config=run_config
    )

    print("")
    print("Estimator Type: {}".format(type(estimator)))
    print("")
    return estimator

# ##########################准备试验数据#############################


#  #####定义试验参数#######
EVAL_AFTER_SEC = 15
NUM_EPOCHS = 1000
BATCH_SIZE = 500
TRAIN_SIZE = 12000
TOTAL_STEPS = (TRAIN_SIZE/BATCH_SIZE)*NUM_EPOCHS
hparams = tf.contrib.training.HParams(
    num_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    max_steps=TOTAL_STEPS,
    hidden_units=[8, 4],
    dropout_prob=0.0)

model_dir = 'trained_models/{}'.format(MODEL_NAME)

run_config = tf.estimator.RunConfig(
    save_checkpoints_steps=480, # to evaluate after each 20 epochs => (12000/500) * 20
    tf_random_seed=19830610,
    model_dir=model_dir
)

print("Model directory: {}".format(run_config.model_dir))
print("Hyper-parameters: {}".format(hparams))

#  #####定义服务函数，用于配置导出 #######
def csv_serving_input_fn()->tf.estimator.export.ServingInputReceiver:
    SERVING_HEADER = ['x', 'y', 'alpha', 'beta']
    SERVING_HEADER_DEFAULTS = [[0.0], [0.0], ['NA'], ['NA']]
    rows_string_tensor = tf.placeholder(dtype=tf.string, shape=[None], name='csv_rows')

    receiver_tensor = {'csv_rows': rows_string_tensor}

    row_columns = tf.expand_dims(rows_string_tensor, -1)
    columns = tf.decode_csv(row_columns, record_defaults=SERVING_HEADER_DEFAULTS)
    features = dict(zip(SERVING_HEADER, columns))

    if PROCESS_FEATURES:
        features = process_features(features)

    return tf.estimator.export.ServingInputReceiver(features, receiver_tensor)


# 定义训练和验证的输入函数
train_input_fn = lambda: csv_input_fn(
    file_name_pattern=TRAIN_DATA_FILES_PATTERN,
    mode=tf.estimator.ModeKeys.TRAIN,
    num_epochs=hparams.num_epochs,
    batch_size=hparams.batch_size
)

eval_input_fn = lambda: csv_input_fn(
    file_name_pattern=VALID_DATA_FILES_PATTERN,
    mode=tf.estimator.ModeKeys.EVAL,
    num_epochs=1,
    batch_size=hparams.batch_size)


train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn,max_steps=hparams.max_steps,hooks=None)

eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn,
    exporters=[tf.estimator.LatestExporter(name="estimate",serving_input_receiver_fn=csv_serving_input_fn,exports_to_keep=1,as_text=True)],
    steps=None,
    throttle_secs = 15 # evalute after each 15 training seconds!
)

In [61]:

if RESUME_TRAINING:
    print("Removing previous artifacts...")
    shutil.rmtree(model_dir, ignore_errors=True)
else:
    print("Resuming training...")

tf.logging.set_verbosity(tf.logging.INFO)
time_start = datetime.utcnow()
print("Experiment started at {}".format(time_start.strftime("%H:%M:%S")))
print(".......................................")
estimator = create_estimator(run_config, hparams)

tf.estimator.train_and_evaluate(estimator=estimator, train_spec=train_spec, eval_spec=eval_spec)

time_end = datetime.utcnow()
print(".......................................")
print("Experiment finished at {}".format(time_end.strftime("%H:%M:%S")))
print("")
time_elapsed = time_end - time_start
print("Experiment elapsed time: {} seconds".format(time_elapsed.total_seconds()))

Resuming training...
Experiment started at 17:23:22
.......................................
INFO:tensorflow:Using config: {'_model_dir': 'trained_models/reg-model-1_2', '_tf_random_seed': 19830610, '_save_summary_steps': 100, '_save_checkpoints_steps': 480, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000000046145B38>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}

Estimator Type: <class 'tensorflow.python.estimator.canned.dnn.DNN

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2400: trained_models/reg-model-1_2\model.ckpt-2400
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['serving_default', 'regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from trained_models/reg-model-1_2\model.ckpt-2400
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: trained_models/reg-model-1_2\export\estimate\temp-b'1542907456'\saved_model.pbtxt
INFO:tensorflow:global_step/sec: 27.0197
INFO:tensorflow:loss = 63141.137, step = 2401 (3.702 sec)
INFO:tensorflow:global_step/sec: 52.2739
INFO:tensorflow:loss = 75557.89, ste

INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from trained_models/reg-model-1_2\model.ckpt-5280
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: trained_models/reg-model-1_2\export\estimate\temp-b'1542907517'\saved_model.pbtxt
INFO:tensorflow:global_step/sec: 26.9469
INFO:tensorflow:loss = 56561.555, step = 5301 (3.711 sec)
INFO:tensorflow:global_step/sec: 52.2739
INFO:tensorflow:loss = 63207.62, step = 5401 (1.913 sec)
INFO:tensorflow:global_step/sec: 47.5737
INFO:tensorflow:loss = 54876.285, step = 5501 (2.103 sec)
INFO:tensorflow:global_step/sec: 46.6853
INFO:tensorflow:loss = 55742.117, step = 5601 (2.187 sec)
INFO:tensorflow:global_step/sec: 45.2694
INFO:tensorflow:loss = 56980.27, step = 5701 (2.163 sec)
INFO:tensorflow:Saving checkpoints for 5760 into trained_models/reg-model-1_2\model.ckpt.
INFO:tensor

INFO:tensorflow:global_step/sec: 28.2646
INFO:tensorflow:loss = 46518.086, step = 8201 (3.538 sec)
INFO:tensorflow:global_step/sec: 52.4109
INFO:tensorflow:loss = 54789.207, step = 8301 (1.908 sec)
INFO:tensorflow:global_step/sec: 53.8503
INFO:tensorflow:loss = 79668.84, step = 8401 (1.858 sec)
INFO:tensorflow:global_step/sec: 53.0223
INFO:tensorflow:loss = 47431.35, step = 8501 (1.885 sec)
INFO:tensorflow:global_step/sec: 53.7634
INFO:tensorflow:loss = 57433.902, step = 8601 (1.860 sec)
INFO:tensorflow:Saving checkpoints for 8640 into trained_models/reg-model-1_2\model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (15 secs).
INFO:tensorflow:global_step/sec: 45.7247
INFO:tensorflow:loss = 44588.125, step = 8701 (2.187 sec)
INFO:tensorflow:global_step/sec: 49.8505
INFO:tensorflow:loss = 49978.945, step = 8801 (2.006 sec)
INFO:tensorflow:global_step/sec: 52.7148
INFO:tensorflow:loss = 52126.703, step = 8901 (1.898 sec)
INFO:tensorflow:global_step/sec: 46.882

INFO:tensorflow:global_step/sec: 47.3261
INFO:tensorflow:loss = 51277.92, step = 11501 (2.113 sec)
INFO:tensorflow:Saving checkpoints for 11520 into trained_models/reg-model-1_2\model.ckpt.
INFO:tensorflow:Skip the current checkpoint eval due to throttle secs (15 secs).
INFO:tensorflow:global_step/sec: 42.2833
INFO:tensorflow:loss = 40229.016, step = 11601 (2.366 sec)
INFO:tensorflow:global_step/sec: 48.7092
INFO:tensorflow:loss = 50562.1, step = 11701 (2.052 sec)
INFO:tensorflow:global_step/sec: 47.8927
INFO:tensorflow:loss = 48221.45, step = 11801 (2.088 sec)
INFO:tensorflow:global_step/sec: 51.1509
INFO:tensorflow:loss = 49800.598, step = 11901 (1.956 sec)
INFO:tensorflow:Saving checkpoints for 12000 into trained_models/reg-model-1_2\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-22-17:27:47
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from trained_models/reg-model-1_2\m

INFO:tensorflow:global_step/sec: 51.7331
INFO:tensorflow:loss = 42559.56, step = 14601 (1.932 sec)
INFO:tensorflow:global_step/sec: 52.0833
INFO:tensorflow:loss = 38261.516, step = 14701 (1.920 sec)
INFO:tensorflow:global_step/sec: 53.2765
INFO:tensorflow:loss = 44734.516, step = 14801 (1.877 sec)
INFO:tensorflow:Saving checkpoints for 14880 into trained_models/reg-model-1_2\model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-22-17:28:51
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from trained_models/reg-model-1_2\model.ckpt-14880
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-22-17:28:51
INFO:tensorflow:Saving dict for global step 14880: average_loss = 95.523125, global_step = 14880, label/mean = 1.1861019, loss = 47761.562, prediction/mean = 1.7469348
INFO:tensorflow:Saving 'checkpoint_path' su

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-22-17:29:56
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from trained_models/reg-model-1_2\model.ckpt-17760
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-22-17:29:57
INFO:tensorflow:Saving dict for global step 17760: average_loss = 94.065895, global_step = 17760, label/mean = 1.1861019, loss = 47032.95, prediction/mean = 1.7473361
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 17760: trained_models/reg-model-1_2\model.ckpt-17760
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['serving_default', 'regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflo

INFO:tensorflow:Finished evaluation at 2018-11-22-17:30:59
INFO:tensorflow:Saving dict for global step 20640: average_loss = 93.03068, global_step = 20640, label/mean = 1.1861019, loss = 46515.34, prediction/mean = 1.690561
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20640: trained_models/reg-model-1_2\model.ckpt-20640
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['serving_default', 'regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from trained_models/reg-model-1_2\model.ckpt-20640
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: trained_models/reg-model-1_2\ex

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['serving_default', 'regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from trained_models/reg-model-1_2\model.ckpt-23520
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: trained_models/reg-model-1_2\export\estimate\temp-b'1542907922'\saved_model.pbtxt
INFO:tensorflow:global_step/sec: 26.0281
INFO:tensorflow:loss = 54940.9, step = 23601 (3.842 sec)
INFO:tensorflow:global_step/sec: 51.5996
INFO:tensorflow:loss = 41626.156, step = 23701 (1.938 sec)
INFO:tensorflow:global_step/sec: 48.9716
INFO:tensorflow:loss = 40447.62, step = 23801 (2.042

In [59]:
INFO:tensorflow:No assets to write.
No such process

In [60]:
os.getcwd()

'E:\\my_proj\\fog_recognition\\recognition_pre_fog_tf\\notebook'